In [1]:
"""
RS-232 interface to an ASI tiger controller.

Hazen 05/18
"""
import traceback
import serial
import time


class RS232(object):
    """
    The basic RS-232 communication object which is used by all the objects
    that communicate with their associated hardware using RS-232.
    """

    def __init__(self,
                 baudrate = None,
                 encoding = 'utf-8',
                 end_of_line = "\r",
                 port = None,
                 timeout = 1.0e-3,
                 wait_time = 1.0e-2,
                 **kwds):
        """
        port - The port for RS-232 communication, e.g. "COM4".
        timeout - The RS-232 time out value.
        baudrate - The RS-232 communication speed, e.g. 9800.
        end_of_line - What character(s) are used to indicate the end of a line.
        wait_time - How long to wait between polling events before it is decided 
                    that there is no new data available on the port. 
        """
        super().__init__(**kwds)
        self.encoding = encoding
        self.end_of_line = end_of_line
        self.live = True
        self.wait_time = wait_time
        try:
            self.tty = serial.Serial(port, baudrate, timeout = timeout)
            self.tty.flush()
            time.sleep(self.wait_time)
        except serial.serialutil.SerialException as e:
            print("RS232 Error:", type(e), str(e))
            self.live = False

    def commWithResp(self, command):
        """
        Send a command and wait (a little) for a response.
        """
        self.sendCommand(command)
        time.sleep(10 * self.wait_time)
        response = ""
        response_len = self.tty.inWaiting()
        while response_len:
            response += self.read(response_len)
            time.sleep(self.wait_time)
            response_len = self.tty.inWaiting()
        if len(response) > 0:
            return response

    def getResponse(self):
        """
        Wait (a little) for a response.
        """
        response = ""
        response_len = self.tty.inWaiting()
        while response_len:
            response += self.read(response_len)
            time.sleep(self.wait_time)
            response_len = self.tty.inWaiting()
        if len(response) > 0:
            return response

    def getStatus(self):
        """
        Return True/False if the port open and can we talk to the hardware.
        """
        return self.live

    def read(self, response_len):
        response = self.tty.read(response_len)
        return response.decode(self.encoding)

    def readline(self):
        response = self.tty.readline()
        return response.decode(self.encoding).strip()
        
    def sendCommand(self, command):
        self.tty.flush()
        self.write(command + self.end_of_line)

    def shutDown(self):
        """
        Closes the RS-232 port.
        """
        if self.live:
            self.tty = None

    def waitResponse(self, end_of_response = False, max_attempts = 200):
        """
        Waits much longer for a response. This is the method to use if
        you are sure that the hardware will respond eventually. If you
        don't set end_of_response then it will automatically be the
        end_of_line character, and this will return once it finds the
        first end_of_line character.
        """
        if not end_of_response:
            end_of_response = str(self.end_of_line)
        attempts = 0
        response = ""
        index = -1
        while (index == -1) and (attempts < max_attempts):
            response_len = self.tty.inWaiting()
            if response_len > 0:
                response += self.read(response_len)
            time.sleep(self.wait_time)
            index = response.find(end_of_response)
            attempts += 1
        return response

    def write(self, string):
        self.tty.write(string.encode(self.encoding))

    def writeline(self, string):
        msg = string + self.end_of_line
        self.tty.write(msg.encode(self.encoding))


class Tiger(RS232):
    """
    Tiger controller interface class.
    """
    def __init__(self, **kwds):
        """
        Connect to the tiger controller at the specified port.
        """
        self.live = True
        self.unit_to_um = 0.1
        self.um_to_unit = 1.0/self.unit_to_um

        # FIXME: Why are we storing the position?
        self.x = 0
        self.y = 0
        self.z = 0

        # Try and connect to the controller.
        try:
            super().__init__(**kwds)
            assert not (self.commWithResp("WHO") == None)
            self.joystickOnOff(True)#BB edit

        except (AttributeError, AssertionError):
            print(traceback.format_exc())
            self.live = False
            print("Tiger controller is not connected? Controller is not on?")
            print("Failed to connect to the tiger controller at port", kwds["port"])

    def goAbsolute(self, x, y):
        self.commWithResp("M X={0:.1f} Y={1:.1f}".format(x * self.um_to_unit, y * self.um_to_unit))

    def goRelative(self, x, y):
        self.commWithResp("R X={0:.1f} Y={1:.1f}".format(x * self.um_to_unit, y * self.um_to_unit))

    def jog(self, x_speed, y_speed):
        # Stage velocities are in units of mm/sec.
        vx = x_speed * self.um_to_unit * 1.0e-3
        vy = y_speed * self.um_to_unit * 1.0e-3
        self.commWithResp("VE X={0:.3f} Y={1:.3f}".format(vx, vy))

    def joystickOnOff(self, on):
        # This also turns off the stage motors to disable position
        # feedback control during movies.
        if on:
            try:
                self.commWithResp("J X+ Y+")
                self.commWithResp("MC X+ Y+ Z+")
            except:
                print("Failed communication...")
        else:
            try:
                self.commWithResp("J X- Y-")
                self.commWithResp("MC X- Y- Z-")
            except:
                print("Failed communication...")

    def position(self):
        try:
            [self.x, self.y] = map(lambda x: float(x)*self.unit_to_um, 
                                   self.commWithResp("W X Y").split(" ")[1:3])
        except:
            [self.x, self.y] = [0,0]#BBedit
        return {"x" : self.x,
                "y" : self.y}

    def setLED(self, address, channel, power):
        self.commWithResp(address + "LED " + channel + "={0:0d}".format(int(power)))

    def setTTLMode(self, address, mode):
        self.commWithResp(address + "TTL X={0:0d}".format(int(mode)))

    def setVelocity(self, x_vel, y_vel):
        """
        Set the maximum X/Y speed in mm/sec.
        """
        self.commWithResp("S X={0:.2f} Y={1:.2f}".format(x_vel, y_vel))

    def zero(self):
        self.commWithResp("H X Y")

    def zConfigurePiezo(self, axis, mode):
        self.commWithResp("PM " + str(axis) + "=" + str(mode))
        
    def zMoveTo(self, z):
        """
        Move the z stage to the specified position (in microns).
        """
        self.commWithResp("M Z={0:.2f}".format(z * self.um_to_unit))

    def zPosition(self):
        """
        Query for current z position in microns.
        """
        new_z = self.z
        try:
            temp = self.commWithResp("W Z")
            new_z = float(temp.split(" ")[1])*self.unit_to_um
        except ValueError:
            print("Tiger.zPosition(): could not parse -", temp, "-")            
        self.z = new_z
        return {"z" : self.z}

    def zSetVelocity(self, z_vel):
        """
        Set the maximum Z speed in mm/sec.
        """
        self.commWithResp("S Z={0:.2f}".format(z_vel))
        
    def zZero(self):
        self.commWithResp("H Z")
    def goZRelative(self, z):
        self.commWithResp("R Z={0:.1f}".format(z * self.um_to_unit))


In [2]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.


In [2]:
stage = Tiger(port = "COM3", baudrate = 115200)

In [5]:
stage.commWithResp("J *?")

':A X=2 Y=3 Z=22 O=0 P=23 E=0 \r\n'

In [6]:
stage.commWithResp("J O=23")

':A \r\n'

In [7]:
stage.commWithResp("HERE O=1")

':A\r\n'

In [8]:
stage.commWithResp("H O+")

':A\r\n'